
## 8. Dataset Propio
De las características de altura, peso y talla, realice su propio dataset determinando cuál será su clase. Realice el cálculo de la entropía y ganancia de información.


In [21]:
import pandas as pd

# Crear un dataset con 20 entradas
data = {
    'Altura': [150, 155, 160, 165, 170, 175, 180, 185, 160, 170,
               175, 180, 165, 170, 155, 160, 185, 180, 175, 190],
    'Peso': [50, 55, 60, 65, 70, 75, 80, 85, 70, 75,
             80, 85, 65, 70, 55, 60, 90, 85, 80, 95],
    'Sexo': ['F', 'F', 'F', 'F', 'M', 'M', 'M', 'M', 'F', 'M',
             'M', 'M', 'F', 'F', 'F', 'M', 'M', 'M', 'M', 'M'],
    'Talla': ['S', 'S', 'M', 'M', 'L', 'L', 'XL', 'XL', 'M', 'L',
              'L', 'XL', 'S', 'M', 'S', 'M', 'XL', 'XL', 'L', 'XL'],
    'Edad': [22, 25, 30, 28, 32, 35, 29, 27, 24, 31,
             22, 30, 26, 31, 29, 28, 34, 30, 36, 25]
}

# Convertir a DataFrame
df = pd.DataFrame(data)

# Convertir altura a metros y calcular IMC
df['Altura_m'] = df['Altura'] / 100  # Altura en metros
df['IMC'] = df['Peso'] / (df['Altura_m'] ** 2)

# Clasificación del IMC
def classify_bmi(imc):
    if imc < 18.5:
        return 'Bajo peso'
    elif imc < 24.9:
        return 'Peso normal'
    elif imc < 29.9:
        return 'Sobrepeso'
    else:
        return 'Obesidad'

df['Clasificación_IMC'] = df['IMC'].apply(classify_bmi)

print(df[['Peso', 'Altura', 'IMC', 'Clasificación_IMC']])


    Peso  Altura        IMC Clasificación_IMC
0     50     150  22.222222       Peso normal
1     55     155  22.892820       Peso normal
2     60     160  23.437500       Peso normal
3     65     165  23.875115       Peso normal
4     70     170  24.221453       Peso normal
5     75     175  24.489796       Peso normal
6     80     180  24.691358       Peso normal
7     85     185  24.835646       Peso normal
8     70     160  27.343750         Sobrepeso
9     75     170  25.951557         Sobrepeso
10    80     175  26.122449         Sobrepeso
11    85     180  26.234568         Sobrepeso
12    65     165  23.875115       Peso normal
13    70     170  24.221453       Peso normal
14    55     155  22.892820       Peso normal
15    60     160  23.437500       Peso normal
16    90     185  26.296567         Sobrepeso
17    85     180  26.234568         Sobrepeso
18    80     175  26.122449         Sobrepeso
19    95     190  26.315789         Sobrepeso


### Entropía
La entropía es una medida de la incertidumbre o desorden en un conjunto de datos. Cuanto mayor sea la entropía, mayor será la diversidad de clases en el conjunto. Se utiliza para cuantificar la cantidad de información necesaria para describir el estado de un sistema.

In [24]:
import numpy as np

def entropy(column):
    probabilities = column.value_counts(normalize=True)
    return -np.sum(probabilities * np.log2(probabilities))

# Función para calcular la entropía de todas las columnas
def calculate_entropies(df):
    entropies = {}
    for column in df.columns:
        entropies[column] = entropy(df[column])
    return entropies

# Calcular y mostrar la entropía de cada columna
entropies = calculate_entropies(df)
for col, ent in entropies.items():
    print(f'Entropía de {col}: {ent:.4f}')



Entropía de Altura: 3.0710
Entropía de Peso: 3.2087
Entropía de Sexo: 0.9710
Entropía de Talla: 1.9855
Entropía de Edad: 3.5842
Entropía de Altura_m: 3.0710
Entropía de IMC: 3.7219
Entropía de Clasificación_IMC: 0.9710


Las variables de **altura, peso, edad y IMC** tienen una alta entropía, lo que sugiere que son útiles para el análisis de salud.
Por otro lado, la baja entropía en sexo y clasificación del IMC indica que estas variables tienen una menor variabilidad y son más fáciles de predecir.







### Ganancia de Información
La ganancia de información mide la reducción de incertidumbre al conocer el valor de un atributo en relación con una clase objetivo. Se calcula como la diferencia entre la entropía original y la entropía condicionada, indicando cuánto aporta un atributo a la predicción de la clase.

In [27]:
# Función para calcular la ganancia de información respecto a la clasificación de IMC
def information_gain(df, attribute, target):
    total_entropy = entropy(df[target])
    attribute_values = df[attribute].value_counts(normalize=True)
    conditional_entropy = 0

    for value in attribute_values.index:
        subset = df[df[attribute] == value]
        conditional_entropy += attribute_values[value] * entropy(subset[target])

    return total_entropy - conditional_entropy

# Calcular ganancia de información para cada columna respecto a 'Clasificación_IMC'
gains_imc = {}
for column in df.columns:
    if column not in ['Clasificación_IMC', 'IMC', 'Altura_m']:  # Excluir columnas no necesarias
        gains_imc[column] = information_gain(df, column, 'Clasificación_IMC')

# Mostrar resultados
for col, gain in gains_imc.items():
    print(f'Ganancia de Información de {col} respecto a Clasificación_IMC: {gain:.4f}')

Ganancia de Información de Altura respecto a Clasificación_IMC: 0.3200
Ganancia de Información de Peso respecto a Clasificación_IMC: 0.4577
Ganancia de Información de Sexo respecto a Clasificación_IMC: 0.1656
Ganancia de Información de Talla respecto a Clasificación_IMC: 0.2722
Ganancia de Información de Edad respecto a Clasificación_IMC: 0.5332



La Ganancia de Información mide qué tanto ayuda una variable a predecir el IMC.

Edad (0.5332) y Peso (0.4577) son las más relevantes para predecir el IMC.
Altura (0.3200) y Talla (0.2722) tienen una relación moderada.
Sexo (0.1656) aporta poca información.


